In [19]:
from pprint import pprint
import jieba
import jieba.analyse
from gensim import corpora
from gensim.models import TfidfModel, LsiModel, LdaModel
from gensim.matutils import corpus2dense, corpus2csc
from sklearn.cluster import KMeans

text_corpus  = [
    '中国人民解放军赢得了抗日战争的胜利',
    '中国人民解放军赢得了朝鲜战争的胜利',
    '八路军进入了抗日战争阶段',
    '印度军在边境与我军发生了冲突',
    '鹿晗是大家喜欢的大众偶像，他会唱歌',
    '韩红歌唱的很不错，但是因为各种原因，长相不好看',
    '好声音里有很多唱歌好的人，其中有解放军',
    '一场歌唱的战争已经打响，大家凭借歌唱实例进行比拼'
]

In [9]:
stopwords = {}.fromkeys([ line.rstrip() for line in open('f:/data/baidu_stopword.txt', 'r', encoding='utf-8') ])
texts = [ [word for word in jieba.lcut(line, cut_all=False) if word not in stopwords] for line in text_corpus]

In [10]:
dictionary = corpora.Dictionary(texts)
bow_corpus =  [dictionary.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)

In [11]:
pprint(bow_corpus)
pprint(tfidf[bow_corpus[1]])

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(0, 1), (2, 1), (3, 1), (4, 1)],
 [(1, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)],
 [(15, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1)],
 [(24, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)]]
[(0, 0.4364357804719848),
 (2, 0.4364357804719848),
 (3, 0.4364357804719848),
 (4, 0.6546536707079772)]


In [12]:
from gensim import similarities
index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)

In [13]:
query_document =  jieba.lcut('韩红是个女的', cut_all=False)
query_bow = dictionary.doc2bow(query_document)
#sims = index[tfidf[query_bow]]
#print(list(enumerate(sims)))

In [14]:
lsi_model = LsiModel(bow_corpus, id2word=dictionary, num_topics=10)  # initialize an LSI transformation
corpus_lsi = lsi_model[bow_corpus]

In [15]:
lsi_model.print_topics(10)

[(0,
  '-0.674*"歌唱" + -0.243*"比拼" + -0.243*"一场" + -0.243*"战争" + -0.243*"实例" + -0.243*"打响" + -0.189*"韩红" + -0.189*"原因" + -0.189*"不错" + -0.189*"不"'),
 (1,
  '0.608*"好" + 0.376*"唱歌" + 0.304*"里" + 0.304*"声音" + 0.304*"解放军" + 0.304*"人" + 0.304*"很多" + 0.072*"大众" + 0.072*"喜欢" + 0.072*"他会"'),
 (2,
  '0.522*"胜利" + 0.522*"赢得" + 0.522*"中国人民解放军" + 0.331*"抗日战争" + 0.256*"朝鲜战争" + 0.065*"八路军" + 0.065*"阶段" + 0.000*"好" + 0.000*"唱歌" + 0.000*"人"'),
 (3,
  '-0.311*"很" + -0.311*"长相" + -0.311*"不错" + -0.311*"原因" + -0.311*"好看" + -0.311*"不" + -0.311*"韩红" + 0.243*"比拼" + 0.243*"战争" + 0.243*"一场"'),
 (4,
  '-0.408*"边境" + -0.408*"发生" + -0.408*"印度" + -0.408*"我军" + -0.408*"军在" + -0.408*"冲突" + 0.000*"他会" + 0.000*"鹿晗" + 0.000*"偶像" + 0.000*"大众"'),
 (5,
  '0.405*"鹿晗" + 0.405*"偶像" + 0.405*"他会" + 0.405*"喜欢" + 0.405*"大众" + 0.310*"唱歌" + -0.191*"好" + -0.096*"很多" + -0.096*"解放军" + -0.096*"声音"'),
 (6,
  '-0.588*"抗日战争" + -0.530*"八路军" + -0.530*"阶段" + 0.194*"朝鲜战争" + 0.136*"赢得" + 0.136*"中国人民解放军" + 0.136*"胜利" + 0.000*"好" + 0.000*"唱歌" +

In [16]:
for doc, as_text in zip(corpus_lsi, texts):
    print(doc, as_text)

[(2, 1.8962320505692822), (6, -0.178956254757758), (7, -0.6101464326511241)] ['中国人民解放军', '赢得', '抗日战争', '胜利']
[(2, 1.8211405849713074), (6, 0.6022969864340928), (7, 0.5662908350863342)] ['中国人民解放军', '赢得', '朝鲜战争', '胜利']
[(2, 0.45983757928433056), (6, -1.6473749306407555), (7, 0.2733225906768318)] ['八路军', '抗日战争', '阶段']
[(4, -2.4494897427831783)] ['印度', '军在', '边境', '我军', '发生', '冲突']
[(1, 0.7350681769402295), (5, 2.336594696401103)] ['鹿晗', '喜欢', '大众', '偶像', '他会', '唱歌']
[(0, -2.000000000000001), (3, -1.9999999999999978)] ['韩红', '歌唱', '很', '不错', '原因', '长相', '不', '好看']
[(1, 3.113798765615652), (5, -0.5515951842161445)] ['好', '声音', '里', '很多', '唱歌', '好', '人', '解放军']
[(0, -2.5615528128088285), (3, 1.5615528128088334)] ['一场', '歌唱', '战争', '打响', '歌唱', '实例', '比拼']


In [30]:
lda = LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=4)
topic_list = lda.print_topics(4)
print("10个主题的单词分布为：\n")
for topic in topic_list:
    print(topic)

10个主题的单词分布为：

(0, '0.109*"歌唱" + 0.061*"胜利" + 0.061*"战争" + 0.061*"实例" + 0.061*"一场" + 0.061*"打响" + 0.061*"中国人民解放军" + 0.060*"比拼" + 0.060*"赢得" + 0.060*"朝鲜战争"')
(1, '0.135*"抗日战争" + 0.075*"赢得" + 0.075*"中国人民解放军" + 0.075*"胜利" + 0.075*"八路军" + 0.074*"阶段" + 0.016*"歌唱" + 0.015*"朝鲜战争" + 0.015*"鹿晗" + 0.015*"唱歌"')
(2, '0.088*"好" + 0.049*"唱歌" + 0.049*"歌唱" + 0.049*"解放军" + 0.049*"韩红" + 0.049*"很" + 0.049*"不错" + 0.049*"声音" + 0.049*"长相" + 0.049*"人"')
(3, '0.058*"唱歌" + 0.058*"他会" + 0.058*"我军" + 0.058*"边境" + 0.058*"偶像" + 0.058*"发生" + 0.058*"冲突" + 0.058*"印度" + 0.058*"大众" + 0.058*"军在"')


In [31]:
corpus_lda = lda[bow_corpus]
for a in corpus_lda:
    print(max(a,  key = lambda k: k[1]))

(1, 0.84759825)
(0, 0.84574276)
(1, 0.81235063)
(3, 0.89272004)
(3, 0.8924069)
(2, 0.9158906)
(2, 0.9162965)
(0, 0.905952)
